У меня пока что очень мало репрезентативного кода, но я горжусь функцией ниже.

Это функция для подготовки датасета из первого проекта специализации,
и я ускорил её на датасете из 150 пользователей с ~31 минуты до ~2 секунд.

Меня очень порадовал этот результат, и поэтому я выкладываю обе версии сюда.

In [1]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import os
import numpy as np
import pandas as pd
from sklearn.externals import joblib

from time import time

In [2]:
PATH_TO_DATA = '.' 

In [3]:
#~31 min version
#under construction
#need a file from home

In [4]:
#~2 sec version
def prepare_train_set_new(path_to_csv_files, session_length=10):   
    tme = time()
    uid = 1
    u_sites = dict()
    compact_frame = []
    
    framepaths = glob(path_to_csv_files + os.sep + '*.csv')
    for framepath in framepaths:
        frame = pd.read_csv(framepath)
        frame = list(frame['site'].values)
        for i in range(0, len(frame), session_length):
            row = frame[i : i + session_length]
            for key in row:
                u_sites[key] = u_sites.get(key, 0) + 1
            k = session_length - len(row)
            if k > 0:
                zeros = [0] * k
                row.extend(zeros)
            row.append(uid)
            compact_frame.append(row)
        uid += 1
    
    u_sites = {kv[0]: (i, kv[1]) for i, kv in enumerate(sorted(u_sites.items(), 
                                                      key=lambda item: item[1], 
                                                      reverse=True), start=1)}

    columns = ['site' + str(i) for i in range(1, session_length + 1)]
    columns.append('user_id')
    
    compact_frame = pd.DataFrame(compact_frame, columns=columns)
    
    for item in columns[:-1]:
        compact_frame[item] = compact_frame[item].apply(lambda x: u_sites.get(x, (0, 0))[0])
    print('\n', time() - tme, sep='')     
    return compact_frame, u_sites


In [5]:
#%prun       tme = time()      print('\n', time() - tme, sep='')     
dff, uniques = prepare_train_set_new(os.path.join(PATH_TO_DATA, '10users'), 10)


0.20548367500305176


In [6]:
dff, uniques = prepare_train_set_new(os.path.join(PATH_TO_DATA, '3users'), 10)


0.009976387023925781


In [7]:
#%%time
train_data_150users, site_freq_150users = prepare_train_set_new(os.path.join(PATH_TO_DATA, '150users'), 
                                                     session_length=10)


2.3178670406341553


In [20]:
#%prun 
dff, uniques = prepare_train_set(os.path.join(PATH_TO_DATA, '10users'), 10)

AttributeError: 'str' object has no attribute 'append'

In [11]:
dff, uniques = prepare_train_set(os.path.join(PATH_TO_DATA, '3users'), 10)


0.01196908950805664


In [12]:
#%%time
train_data_150users, site_freq_150users = prepare_train_set(os.path.join(PATH_TO_DATA, '150users'), 
                                                     session_length=10)


2.2659730911254883
